pip install pymssql 



In [1]:
import pandas as pd
import pymssql 

In [2]:
server="SQL-retail2.nikamed.local"
base="Retail2_shops"

In [31]:
sql="""
with mag_iskl as (SELECT CAST(_IDRRef AS uniqueidentifier) AS Ссылка,
_IDRRef, 
_Code AS Код
FROM _Reference10
where _Code='0-002')

SELECT 
cast(dateadd(year, -2000, i._fld10076) as date) AS date,
sum(_fld10077) AS Трафик,
cast( r._Fld2907RRef as uniqueidentifier) AS cсМагазин

FROM _InfoRg10074 i 


left outer JOIN _reference142 r ON i._Fld10075RRef = r._IDRRef
left outer join mag_iskl mi on r._Fld2907RRef=mi._IDRRef

WHERE i._fld10076 > '08.01.4021'
AND _fld10077 > 0
and mi._IDRRef is null
group by cast(dateadd(year, -2000, i._fld10076) as date) , cast( r._Fld2907RRef as uniqueidentifier)
"""

In [32]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,charset =  'cp1251',database = base ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

conn.close()

In [33]:
df_tr=read_sql(sql,base, server)


In [34]:
sum(df_tr['Трафик'])

684464.0

In [35]:
df_tr


,date,Трафик,cсМагазин
0,2021-09-09,56.0,4a3cfa88-7492-28df-11e3-f1630a26cc55
1,2021-09-25,22.0,4a3c3d86-7492-28df-11e8-75f181037b39
2,2021-09-17,52.0,11008787-72d8-8824-11de-d377741509e2
3,2021-08-18,66.0,4a3ca8ad-7492-28df-11e7-28d16a5e69b0
4,2021-09-18,126.0,4a3caaad-7492-28df-11e7-6bf4b33a5c97
...,...,...,...
11239,2021-08-27,62.0,1500968d-fa5d-1787-11e9-f5a359bc4756
11240,2021-08-07,30.0,1100b287-72d8-8824-11df-10ed9da89bc6
11241,2021-08-23,83.0,4a3ca086-7492-28df-11e1-26e56017ec33
11242,2021-08-21,24.0,1500708d-015d-0b43-11e9-db79936180ae


In [63]:
sql="""



select top(1000)
       cast(dateadd(year, -2000, d21._date_time) as date) as 'Дата' , 
         
       
       iif(e._EnumOrder=1,'ВозвратДВД','Продажи') as 'ВидОперации'
       ,iif(e._EnumOrder=1, -d21t._Fld6384, d21t._Fld6384 ) as 'Количество'
       ,iif(e._EnumOrder=1, -d21t._Fld6397, d21t._Fld6397) as 'Сумма'
        
              , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
        
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), 
                                  cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
        

from _Document21_VT6378 d21t

left outer join _Document21 d21 on d21t._Document21_IDRRef = d21._IDRRef /*исх чек*/

LEFT OUTER JOIN _reference47 r47 ON d21._Fld6331RRef = r47._idrref /*вид документа*/
LEFT OUTER JOIN _Enum352 e ON d21._Fld6333RRef = e._idrref /*вид документа в перечислениях чека ккм - 0-продажа, 1-возврат*/

left outer join _Reference10 r10 on d21._Fld6339RRef = r10._idrref /*shop*/


left outer join  _Document21 d2_21 on d21._Fld6353RRef=d2_21._idrref /*поиск мотив маг*/
left outer join _Reference10 r2_10 on d2_21._Fld6339RRef = r2_10._idrref /*shop*/

where d21._marked = 0 and d21._Posted = 1
and d21._date_time >= '08.01.4021' 


union all

select top(1000)
       cast(dateadd(year, -2000, d._date_time) as date) as 'Дата', 
       
        N'Возврат'
       , -dt._Fld3984 as 'Количество'
       , -dt._Fld3988 as 'СуммаЧека'
       
       , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
        
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
        

from _Document216_VT3980 dt

left outer join _Document216 d on dt._Document216_IDRRef = d._IDRRef /*исх возвратный чек*/

left outer join _Reference10 r10 on d._Fld3950RRef = r10._idrref /*shop*/
left outer join  _Reference172 r172  on dt._Fld4001RRef = r172._idrref
left outer join  _Reference23 r23  on dt._Fld3982RRef = r23._idrref


left outer join _Reference131 r131 on d._Fld11137RRef= r131._idrref --  склады
left outer join _Reference10 r2_10 on r131._Fld2686RRef = r2_10._idrref /*shop к складу*/

where d._marked = 0 and d._Posted = 1
and d._date_time >= '08.01.4021'


"""

In [64]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,database = base ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

In [65]:
%%time
df=read_sql(sql,base, server)

Wall time: 3.16 s


In [66]:
sum(df['Сумма'])


-1094412.08

In [67]:
df


,Дата,ВидОперации,Количество,Сумма,ccМагазин,ccМагазинМотивационный
0,2021-09-09,Продажи,1.0,2050.0,4a3c979d-7492-28df-11e1-74dcac7abe53,4a3c979d-7492-28df-11e1-74dcac7abe53
1,2021-09-02,Продажи,1.0,2495.0,4a3c0284-7492-28df-11e2-35743d65c730,4a3c0284-7492-28df-11e2-35743d65c730
2,2021-08-06,Продажи,1.0,9093.0,4a3c0284-7492-28df-11e2-35743d65c730,4a3c0284-7492-28df-11e2-35743d65c730
3,2021-08-04,Продажи,1.0,2331.0,4a3c0284-7492-28df-11e2-35743d65c730,4a3c0284-7492-28df-11e2-35743d65c730
4,2021-09-10,Продажи,1.0,1521.0,4a3c0284-7492-28df-11e2-35743d65c730,4a3c0284-7492-28df-11e2-35743d65c730
...,...,...,...,...,...,...
1995,2021-08-20,Возврат,-1.0,-3367.0,4a3cafb9-7492-28df-11e7-203689cf9bf9,4a3cafb9-7492-28df-11e7-203689cf9bf9
1996,2021-08-25,Возврат,-1.0,-334.5,4a3ce78c-7492-28df-11e6-f8d0f307b036,4a3ce78c-7492-28df-11e6-f8d0f307b036
1997,2021-08-07,Возврат,-1.0,-1947.0,4a3cecab-7492-28df-11e0-fa57cdfd7b0a,4a3cecab-7492-28df-11e0-fa57cdfd7b0a
1998,2021-09-28,Возврат,-1.0,-3901.0,4a3cecab-7492-28df-11e0-fa57cdfd7b0a,4a3cecab-7492-28df-11e0-fa57cdfd7b0a
